In [1]:
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d
from numpy import *
import matplotlib.pyplot as plt
import scipy.integrate as integrate
import scipy.special as special
from datetime import datetime
from scipy.stats import norm
from scipy import optimize

In [2]:
def yearfraction(tenor):
    size = len(tenor)
    sizevalue = size -1
    value = (int)(tenor[:sizevalue])
    unit = tenor[sizevalue]
    
    if unit == 'D' or unit =='d': 
        return value/365
    if unit == 'W' or unit =='w':
        return value*7/365
    if unit == "M" or unit == "m":
        return value/12
    if unit =="y" or unit =="Y": 
        return value
    
    print("Problème ")
    return 0

In [3]:
asof = np.datetime64("2018-11-09")

In [4]:
Tenor_D = np.array([yearfraction(u) for u in ["1D","1Y","2Y","4Y","5Y"]])
ZC_d = np.array([0.0001,0.01,0.01,0.01,0.01])
curve_d = interp1d(Tenor_D,ZC_d,kind='linear',fill_value="extrapolate")

Tenor_F = Tenor_D
ZC_f = np.array([0.0001,0.01,0.01,0.01,0.01])
curve_f = interp1d(Tenor_F,ZC_f,kind='linear',fill_value="extrapolate")

df = pd.DataFrame()
df["1Y"] = [0.01,0.01,0.01]
df["2Y"] = [0.01,0.01,0.01]
df["5Y"] = [0.01,0.01,0.01]
df["Swap_lenght"] = [yearfraction(u) for u in ["5Y","10Y","15Y"]] 
df = df.set_index("Swap_lenght")
df.columns = [yearfraction(u) for u in df.columns.values]

surfaceVol = df
impliedSwapTenors = np.asarray(surfaceVol.index.values)
impliedMaturities = np.asarray(surfaceVol.columns.values)
print(surfaceVol)

                1     2     5
Swap_lenght                  
5            0.01  0.01  0.01
10           0.01  0.01  0.01
15           0.01  0.01  0.01


In [5]:
def getPosInterpol(obj, t):
    posSupTuple = np.where(obj > t)
    posSub = 0
    
    
    if len(posSupTuple[0]) == 0:
        posSub = len(obj)-1
    else:
        posSub = posSupTuple[0][0]
    
    posInf = posSub-1
    
    if posInf < 0:
        posInf = 0
        
    return (posInf,posSub)
    
def linearInterp(yb,ya,xb,xa,x):
    a = (yb-ya)/(xb-xa)
    return a*(x-xa)+ya

def getImpliedATMVol(matu,tenorSwap):
    posTenor = getPosInterpol(impliedSwapTenors,tenorSwap)
    posMatu = getPosInterpol(impliedMaturities,matu)
    
    tenor1 = impliedSwapTenors[posTenor[0]]
    tenor2 = impliedSwapTenors[posTenor[1]]
    
    mat1 = impliedMaturities[posMatu[0]]
    mat2 = impliedMaturities[posMatu[1]]
    
    vol_tenor1_matu1 = surfaceVol.loc[tenor1,mat1]
    vol_tenor2_matu1 = surfaceVol.loc[tenor2,mat1]
    vol_tenor1_matu2 = surfaceVol.loc[tenor1,mat2]
    vol_tenor2_matu2 = surfaceVol.loc[tenor2,mat2]
    
    if tenor1 == tenorSwap and mat1 == matu:
        return vol_tenor1_matu1
    
    if tenor1 == tenorSwap:
        return linearInterp(vol_tenor1_matu2,vol_tenor1_matu1,mat2,mat1,matu)
    
    if mat1 == matu:
        return linearInterp(vol_tenor2_matu1,vol_tenor1_matu1,tenor2,tenor1,tenorSwap)
    
    M = [1.0]*(4)*(4)
    M = reshape(M,(4,4))
    
    M[0][0] = tenor1
    M[1][0] = tenor1
    M[2][0] = tenor2
    M[3][0] = tenor2
    M[0][1] = mat1
    M[1][1] = mat2
    M[2][1] = mat1
    M[3][1] = mat2
    M[0][2] = mat1*tenor1 
    M[1][2] = mat2*tenor1 
    M[2][2] = mat1*tenor2 
    M[3][2] = mat2*tenor2
    
    f = np.zeros(4)
    f[0] = vol_tenor1_matu1
    f[1] = vol_tenor1_matu2
    f[2] = vol_tenor2_matu1
    f[3] = vol_tenor2_matu2
    trsM = np.transpose(M)
    cov = np.matmul(trsM,M)
    d = np.matmul(trsM,f)
    res = np.matmul(np.linalg.inv(cov),d)
    f[0] = tenorSwap
    f[1] = matu
    f[2] = tenorSwap*matu
    f[3] = 1
    #print(res)
    return np.matmul(res,f)
    #return 0.0

In [6]:
def getImpliedVol(mat,tenor5wap,K):
    return getImpliedATMVol(mat,tenor5wap)

In [7]:
def transformDatesToYF(vectDates, daycount):
    listRes = []
    i = 0
    size = len(vectDates)
    #print(vectDates)
    while i < size:
        tmpDate = vectDates[i]    
        if tmpDate > asof:
            dt = (tmpDate-asof).astype('timedelta64[D]')/ np.timedelta64(1, 'D')    
            if(daycount == "ACT365FIXED"):
                listRes.append(dt/365.0)
        i = i+1
    return np.asarray(listRes)

In [8]:
#bermuda Caractéristiques
tenorLibor = yearfraction("6M")
fixedFrequency = yearfraction("12M")
sens = 1
L = 0.025
Times = np.zeros(len(impliedMaturities)+1)
Times[1:] = impliedMaturities
Vols = np.zeros(len(impliedMaturities)+1)
maxVol = 0.9

In [9]:
def initializeVol(ts, vs):
    Times[:] = ts
    Vols[:] = vs

def setVol(posl, value):
    Vols[posl] = value

def checkVol(x):
    if 0<x<maxVol:
        return True
    return False

def getVol(t):
    vols = Vols
    nR = Times.size
    i = 0
    if t == 0:
        return 0.0
    j = 0
    
    while j < nR and t >= Times[j]:
        j =j+1
    
    if t == Times[j-1]:
        vol = vols[j-1]
        return vol
    
    if j >= nR:
        vol = vols[j-1]
        return vol
    
    if j > 0:
        vol = vols[j]
        return vol
    
    return 0.0

def getB(t,T):
    return getB_withL(t,T,L)

def getB_withL(t,T,L):
    if L > 0:
        return (1.0-np.exp(-L*(T-t)))/L
    if L == 0:
        return T-t

In [10]:
#dXt = b(t,Xt)dt+sig(t)Xt)dwt
def integrandeVol(i,j,Tm,g):
    vols = Vols
    #print(voLs)
    nR = Times.size
    ssum = 0
    t = Times[i]
    T = Times[j]
    
    if i >=nR:
        #ExtrapoLation
        vol = vols[nR-1]
        tj = t
        tjj = T
        ssum = ssum + (vol * vol * g(t,T,Tm,tj,tjj))
        #print(voL)
        return ssum
    
    tj = t
    if i == j:
        tjj = T
    else:
        tjj = Times[i]
    
    vol = vols[i]
    ssum = ssum + (vol * vol * g(t,T,Tm,tj,tjj))
    
    if i == j:
    #print(voL)
        return ssum
    
    K = i
    while K < j - 1:
        tj = Times[K]
        tjj = Times[K + 1]
        vol = vols[K + 1]
        ssum = ssum + (vol * vol * g(t,T,Tm,tj,tjj))
        K = K + 1
        
    if j >= nR:
        tj = Times[nR-1]
        vol = vols[nR-1]
    else:
        tj = Times[K]
        vol = vols[K + 1]
        
    tjj = T
    #print(voL)
    ssum = ssum + (vol * vol * g(t,T,Tm,tj,tjj))
    return ssum

def integrandeVol(t,T,Tm,g):
    vols = Vols
    #print(voLs)
    nR = Times.size
    ssum = 0
    i = 0
    
    while i < nR and t >= Times[i] :
        i = i + 1
    j = i
    if i >=nR:
        #ExtrapoLation
        vol = vols[nR-1]
        tj = t
        tjj = T
        ssum = ssum + (vol * vol * g(t,T,Tm,tj,tjj))
        #print(voL)
        return ssum
    
    while j < nR and T >= Times[j]:
        j = j + 1
    
    tj = t
    if i == j :
        tjj = T
    else:
        tjj = Times[i]
        
    vol = vols[i]
    ssum = ssum + (vol * vol * g(t,T,Tm,tj,tjj))
    
    if i == j:    
        return ssum
    
    K = i
    while K < j - 1:
        tj = Times[K]
        tjj = Times[K + 1]
        vol = vols[K + 1]
        ssum = ssum + (vol * vol * g(t,T,Tm,tj,tjj))
        K = K + 1
        
    if j >= nR:
        tj = Times[nR-1]
        vol = vols[nR-1]
    else:
        tj = Times[K]
        vol = vols[K + 1]
    tjj = T
    #print(voL)
    ssum = ssum + (vol * vol * g(t,T,Tm,tj,tjj))

    return ssum

def subVariance(t,tt,tk, tj,tjj):
    return (np.exp(-2 * L * tt) / (2 * L)) * (np.exp(2 * L * tjj)-np.exp(2 * L * tj))

def getVariance(t, tt):
    return integrandeVol(t,tt,t,subVariance)

In [13]:
def subMu(s,t,Tm, tj,tjj):
    coeff1 = np.exp(-L*(Tm+t))*0.5
    tmp1 = coeff1*(np.exp(2*L*tjj)-np.exp(2*L*tj))
    tmp2 = np.exp(-L*t)*(np.exp(L*tjj)-np.exp(L*tj))
    return tmp1-tmp2

def Mu(s,t,Tm):
    return (1/L**2)*integrandeVol(s,t,Tm,subMu)

def subft(t,T,Tk,tj,tjj):
    coeff = 0.5*np.exp(-L*T)
    return np.exp(L*tjj)-np.exp(L*tj)-coeff*(np.exp(2*L*tjj)-np.exp(2*L*tj))

def getDF_d(t):
    zc = float(curve_d(t))
    return np.exp(-zc*t)

def getDF_f(t):
    zc = float(curve_f(t))
    return np.exp(-zc*t)


def getF_init_t(t):    
    e = 1e-2
    dt = e
    dft = getDF_d(t)
    dfte = getDF_d(t+e)
    return (np.log(dft/dfte))/dt
   
def ft(t):
    coeff = np.exp(-L*t)/(L*L)
    test = coeff*integrandeVol(0,t,t,subft)
    vol = Vols[1]
    testCible = (vol*vol/(2*L*L))*(1-np.exp(-L*t))*(1-np.exp(-L*t))
    #print(test)
    #print(testCible)
    return getF_init_t(t)+test

def subV(t,T,Tk, tj,tjj):
    term1 = (1/(2*L))*(np.exp(-2*L*(T-tjj)) - np.exp(-2*L*(T-tj))) 
    term2 = (1/(2*L))*(np.exp(-2*L*(Tk-tjj)) - np.exp(-2*L*(Tk-tj))) 
    term3 = (1/(L))*(np.exp(-L*(T-tjj)) - np.exp(-L*(T-tj)))
    term4 = (1/(L))*(np.exp(-L*(Tk-tjj)) - np.exp(-L*(Tk-tj))) 
    return term1 - term2 + 2*(term4 - term3)

def V(t,T):
    return (1/L**2)*integrandeVol(0,t,T,subV)
    
    
def getDF(t,x,T):
    if t == 0 :
        return getDF_d(T)
    if t == T:
        return 1
    
    df_0_T = getDF_d(T)
    df_0_t = getDF_d(t)
    cc = (1-np.exp(-L*(T-t)))/L
    
    return (df_0_T/df_0_t)*np.exp(-cc*x+0.5*V(t,T))

def getDF_L(t,x,T):
    df_d_t = getDF_d(t) 
    df_d_T = getDF_d(T) 
    df_f_t = getDF_f(t) 
    df_f_T = getDF_f(T)
    
    return ((df_f_T/df_f_t)*(df_d_t/df_d_T))*getDF(t,x,T)

def getLibor(t,x,tt, ttt):
    accrual = ttt - tt
    if accrual == 0 :
        return 0
    libor = ((getDF_L(t, x, tt) / getDF_L(t, x, ttt)) - 1) * (1 / accrual) 
    return libor


def getVSwap(t,swapStart,swapEnd,K, x):
    startT = np.maximum(t,swapStart)
    sumFloatleg = getDF(t, x, swapStart)-getDF(t, x, swapEnd)
    sumFixedleg = 0
    accrual= 0
    tPrec = swapEnd-fixedFrequency
    tNext = swapEnd
    
    while tPrec >= startT:
        accrual = fixedFrequency
        sumFixedleg = sumFixedleg +accrual * K * getDF(t, x, tNext)
        tNext = tPrec
        tPrec = tPrec - fixedFrequency
    res = sens * (sumFloatleg - sumFixedleg)
    return res

def getSwapAtMAndAnnuity(swapStart,swapEnd):
    x = 0
    t = 0
    startT = np.maximum(t,swapStart)
    sumFloatleg = getDF(t, x, swapStart)-getDF(t, x, swapEnd)
    
    #print("TS and TE")
    #print(swapStart)
    #print(swapEnd)
    #print("DF Start")
    #print(getDF(t, x, swapStart))
    
    #print("DF End")
    #print(getDF(t, x, swapEnd))
    #print(sumFloatleg)
    
    K = 1
    sumFixedleg = 0
    accrual= 0
    tPrec = swapEnd-fixedFrequency
    tNext = swapEnd
    
    while tPrec >= startT:
        accrual = fixedFrequency
        sumFixedleg = sumFixedleg +accrual * K * getDF(t, x, tNext)
        tNext = tPrec
        tPrec = tPrec - fixedFrequency
    
    #print(sumFixedleg)
    
    swaprate = (float)(sumFloatleg/sumFixedleg)
    return (swaprate,sumFixedleg,swapEnd-startT)

def fnSwpation(TO,swapEnd,K, x,mu,var):
    res= getVSwap(TO,TO,swapEnd,K, x)
    if res > 0:
        return norm.pdf(x,mu,np.sqrt(var))*res
    return 0

def getSwaptionPrice(T0,swapEnd,K):
    mu = Mu(0,T0,T0)
    var = getVariance(0,T0)
    result = integrate.quad(lambda x: fnSwpation(T0,swapEnd,K,x,mu,var), mu-5*np.sqrt(var), mu+5*np.sqrt(var))
    
    return result[0]*getDF_d(T0)

def BS(F,K,var,D,sens):
    stdVar = np.sqrt(var)
    dl = ((np.log(F/K))/stdVar)+0.5*stdVar
    d2 - dl - stdVar
    res = 0
    call = D*(F*norm.cdf(d1)-K*norm.cdf(d2))
    res = call
    if sens == -1:
        res = res + D*(F-K)
    return res

In [14]:
def BSSwaptionATM(swapStart,swapEnd):
    TO = swapStart
    res = getSwapAtMAndAnnuity(swapStart,swapEnd)
    F = res[0]
    D = res[1]
    TenorSwap = res[2]
    var = getImpliedVol(TO,TenorSwap,F)
    var = (var**2)*TO
    return D*F*(2*norm.cdf(0.5*np.sqrt(var))-1)

def BSSwaptionATM4Calib(sig,T0,F,D):
    var = sig
    var = (var**2)*T0
    return D*F*(2*norm.cdf(0.5*np.sqrt(var))-1)

def BSSwaption(TO,swapEnd,K):
    res = getSwapAtMAndAnnuity(TO,swapEnd)
    F = res[0]
    D = res[1]
    TenorSwap = res[2]
    var = getImpliedVol(TO,TenorSwap,K)
    return BSSwaption4Calib(TO,K,var,F,D)

def BSSwaption4Calib(T0,K,sig,F,D):
    if F == K:
        return BSSwaptionATM4Calib(sig,T0,F,D)
    
    var = (sig**2)*T0
    return BS(F,K,var,D,sens)
    
def newtonFunctionSolve(x,posIVol,TO,swapEnd,K,cible):
    if x <0:
        return -500000000
    if x > 5:
        return 500000000
    
    setVol(posIVol,x)
    priceModel = getSwaptionPrice(TO,swapEnd,K)
    print("Model Price "+ str(priceModel))
    print("Gap "+str(priceModel-cible))
    return ((priceModel-cible)/cible)*10000

In [15]:
print(Vols)

[0. 0. 0. 0.]


In [16]:
def CalibModelOnCoterminalSwaption(onlyInATM = True,matuMax = 11):
    volsterms = np.zeros(len(impliedMaturities)+1)
    volsterms[1:] = impliedMaturities
    guess = 0.0001
    vols = np.zeros(len(volsterms))+guess
    vols[0] = 0
    strike = 0.001
    Notional = 1
    initializeVol(volsterms,vols)    
    i = 0
    N = len(volsterms)
    term = (int)(N/4)
    TenorSwap = impliedSwapTenors[len(impliedSwapTenors)-1]
    
    for i in range(len(volsterms)-1):        
        T0 = volsterms[i+1]
        TenorSwap = matuMax - T0
        res = getSwapAtMAndAnnuity(T0,T0+TenorSwap)
        
        #break
        
        F = res[0]
        D = res[1]
        TenorSwap = res[2]
        var = 0
        
        if onlyInATM == True:
            strike = F
            
        var = getImpliedVol(T0,TenorSwap,strike)
        print("Strike " + str(strike))
        print("F "+ str(F))
        print("Discount " + str(D))
        
        print("volatility "+ str(var)+"||| for matu "+str(T0))
        cible = Notional*BSSwaption4Calib(T0,strike,var,F,D) 
        print("prix cible "+str(cible)+"II matu "+str(T0)) 
        poslVol = i+1
        
        res = optimize.newton(newtonFunctionSolve, guess, args=(poslVol,T0,T0+TenorSwap,strike,cible))
        print("HW Vol "+str(Vols[i+1])+" matu "+str(T0))
        
        if term > 1:
            if i% term == 0:
                print(str((i/N)*100)+"%")
                
        print("................................")
              
    print("....times & vol after....")
    print(Times)
    print(Vols)
    print("...................")
    return (Times, Vols)

In [17]:
res = CalibModelOnCoterminalSwaption()
print(Times)
print(Vols)

Strike 0.010050167084168065
F 0.010050167084168065
Discount 9.37454050898885
volatility 0.01||| for matu 1.0
prix cible 0.00037586468979956616II matu 1.0
Model Price 0.0003307751872115095
Gap -4.508950258805668e-05
Model Price 0.0006615832108533797
Gap 0.0002857185210538135
Model Price 0.0003758647094844014
Gap 1.968483525054826e-11
HW Vol 0.00011363147454583324 matu 1.0
................................
Strike 0.010050167084168055
F 0.010050167084168055
Discount 8.394341835682095
volatility 0.01||| for matu 2.0
prix cible 0.0004759719692255108II matu 2.0
Model Price 0.0004472939199723981
Gap -2.8678049253112682e-05
Model Price 0.0006851826504918403
Gap 0.00020921068126632952
Model Price 0.00047225816746678407
Gap -3.7138017587267074e-06
Model Price 0.0004755374999230857
Gap -4.3446930242510345e-07
Model Price 0.0004759736047574136
Gap 1.6355319028358434e-09
HW Vol 0.00011379376201725268 matu 2.0
................................
Strike 0.010050167084168064
F 0.010050167084168064
Discoun

In [18]:
#check
onlyInATM = True
matuMax = 11
#TenorSwap = impliedSwapTenors[len(impliedSwapTenors)-1]
for i in range(len(impliedMaturities)):
    T0 = impliedMaturities[i]
    TenorSwap = matuMax - T0
    Notional = 1
    res = getSwapAtMAndAnnuity(T0,T0+TenorSwap)
    F = res[0]
    cible = Notional*BSSwaptionATM(T0,T0+TenorSwap)    
    
    prixModel = Notional*getSwaptionPrice(T0,T0+TenorSwap, F)
    print("Prix model : " +str(prixModel) + "Il Prix marché "+str(cible)+ "|| Gap : "+str(prixModel-cible))

Prix model : 0.0003758647094844014Il Prix marché 0.00037586468979956616|| Gap : 1.968483525054826e-11
Prix model : 0.0004759736047574136Il Prix marché 0.0004759719692255108|| Gap : 1.6355319028358434e-09
Prix model : 0.0004941507204855313Il Prix marché 0.0004941500621378049|| Gap : 6.583477263778087e-10


##### Bermudan Caractéristiques

In [19]:
#bermuda Caractéristiques
#startBerm
##FLoat Information
matuStart = 1
tenorSwaption = 10
libtenor = 0.5
fixtenor = 1
freqExcersise = 1

nbFloat = int((tenorSwaption/libtenor))
Notionals_Float = [1]*nbFloat
leverages = [1]*nbFloat
#daycFloat = 'ACT365FIXED'
#tenorLibor = mktExcel_bermCarac['Tenor_Libor'].dropna().values
spreads = [0.0]*nbFloat

Float_Start_Schedule = [matuStart + u*libtenor for u in range(nbFloat)]
#print(FLoat Start_ScheduLe)
Float_End_Schedule = [matuStart+(u+1)*libtenor for u in range(nbFloat)]
Float_Payment_Schedule = Float_End_Schedule
Float_Fixing_Schedule = Float_Start_Schedule


####Fix Information

nbFix = int((tenorSwaption/fixtenor))
Fix_Start_Schedule = [matuStart + u*fixtenor for u in range(nbFix)]
Fix_End_Schedule = [matuStart + (u+1)*fixtenor for u in range(nbFix)]
Fix_Payment_Schedule = Fix_End_Schedule
K = [0.02]*nbFix
Notionals_Fix = [1]*nbFix
##
bermExcersise = [matuStart+u*freqExcersise for u in range(int(tenorSwaption/freqExcersise))]
startSwapIfExercise = bermExcersise
sens = 1

MeasureT = bermExcersise[len(bermExcersise)-1]

In [20]:
#dXt = b(t,Xt)dt+sig(t,Xt)dwt
def b(t,x):
    #vol = getVol(t)
    return -L*x

def sig(t,x):
    vol = getVol(t)
    return vol

In [21]:
def getVSwap(t,posStartSwap,x):
    sizeFix = len(Fix_Start_Schedule)
    sizeFloat = len(Float_Start_Schedule)
    sumFixedleg = 0
    sumFloatleg = 0
    accrual= 0
    ####pricing fix Leg
    tStart = startSwapIfExercise[posStartSwap]
    posStart = np.where(Fix_Start_Schedule == tStart)[0][0]
    i = posStart
    #print(i)
    while i < sizeFix:
        accrual = Fix_End_Schedule[i]-Fix_Start_Schedule[i]
        timePayment = Fix_Payment_Schedule[i]
        #print(K[i])
        sumFixedleg = sumFixedleg +Notionals_Fix[i]*accrual * K[i]*getDF(t, x, timePayment)
        i = i+1
    ####pricing fix Leg
    posStart = np.where(Float_Start_Schedule == tStart)[0][0]
    i = posStart
    while i < sizeFloat:
        accrual = Float_End_Schedule[i]-Float_Start_Schedule[i]
        timePayment = Float_Payment_Schedule[i]
        timeFixing = Float_Fixing_Schedule[i]
        libor = getLibor(t, x, timeFixing, Float_End_Schedule[i])
        alpha = leverages[i]
        s = spreads[i]

        sumFloatleg = sumFloatleg + Notionals_Float[i]*accrual * (alpha*libor + s) * getDF(t, x, timePayment)
        i = 1+1
        
    res = sens * (sumFloatleg - sumFixedleg)
    #print(res)
    return res

In [22]:
def fnSwaption(TO,posStart5wap,x,mu,var):
    res= getVSwap(TO,posStartSwap,x)
    if res > 0:
        #print(res)
        density = (1/np.sqrt(2*math.pi*var))*np.exp(-((x-mu)**2)/(2*var))
        return density*res
    return 0

def getSwaptionPrice(T0,posStartSwap):
    mu = Mu(0,T0,T0)
    var = getVariance(0,T0)
    result = integrate.quad(lambda x: fn5wpation(TO,posStart5wap,x,mu,var), mu-5*np.sqrt(var), mu+5*np.sqrt(var))
    return result[0]*getDF_d(T0)

def check_V_sub(t,T):
    vol = Vols[1]
    coeff = vol*vol/(L*L)
    return coeff*(T-t-2*getB_withL(t,T,L)+getB_withL(t,T,2*L))
def check_V(t,T):
    return check_V_sub(t,T)-checkV_sub(0,T)+checkV_sub(0,t)

#### EDP

In [23]:
#EDP Param
Kscale = 5
Tgrille = np.max(bermExcersise)
variance_0_tend = getVariance(0,Tgrille)
mu_0_tend_Tmeasure = Mu(0,Tgrille,MeasureT)
xBoundsDown = -Kscale*np.sqrt(variance_0_tend)
xBoundsUp = Kscale*np.sqrt(variance_0_tend)
dt = yearfraction("5D")
#dt = (1/365)/10
#dt = le-6
N = (int)(Tgrille/dt)
dtReliquat = Tgrille-N*dt
if dtReliquat > 0:
    N = N+2
else:
    N = N+1
#print(N)
#N= Len(bermExcersise)
timeDisc = np.zeros(N)

k = N-1
while k > 0:
    timeDisc[k] = Tgrille-(N-1-k)*dt
    k = k-1

listTime = timeDisc.tolist()

for el in bermExcersise:
    listTime.append(el)

listTime.sort()
listTime = list(set(listTime))
listTime.sort()
timeDisc = np.asarray(listTime)
N = len(timeDisc)


Mx = 200
dx = (xBoundsUp)/Mx
x0 = 0
spaceDiscr = np.zeros(2*Mx+1)
posPrice = Mx

j = 0
while j <= Mx:
    spaceDiscr[Mx-j] = j*dx
    j = j+1

j = 0
while j <= Mx:
    spaceDiscr[Mx+j] = -j*dx
    j = j+1

def getSpacePos(i):
    if -Mx <= i <= Mx:
        return Mx-i
    print("error, pos i is not good !")

#### Edp RN Solver

In [24]:
###Settings coeff for cranc Nicholson
#coeff i+1, j
def coeffIPlusOne_J(tj,x):
    vol = sig(tj,x)
    res = -((b(tj,x)/(2*dx))+(vol*vol/(2*dx*dx)))
    return res

#coeff i,j
def coeffI_J(tj,x):
    vol = sig(tj,x)
    res = ((vol*vol/(dx*dx)))+(x+ft(tj))
    #print(res)
    return res

#coeff i-1,j
def coeffIMinusOne_J(tj,x):
    vol = sig(tj,x)
    res = ((b(tj,x)/(2*dx))-(vol*vol/(2*dx*dx)))
    # print(res)
    return res

def alphaMx_J(tj,x):
    return 2*coeffIPlusOne_J(tj,x)+coeffI_J(tj,x)

def betaMx_J(tj,x):
    return coeffIMinusOne_J(tj,x)-coeffIPlusOne_J(tj,x)

def theta_Mx_J(tj,x):
    return coeffIPlusOne_J(tj,x)-coeffIMinusOne_J(tj,x)

def kappa_Mx_J(tj,x):
    return 2*coeffIMinusOne_J(tj,x)+coeffI_J(tj,x)

def getA(tj,tjj):
    triMatj = [0.0]*(2*Mx+1)*(2*Mx+1)
    triMatj = reshape(triMatj,(2*Mx+1,2*Mx+1))
    triMatj_1 = [0.0]*(2*Mx+1)*(2*Mx+1)
    triMatj_1 = reshape(triMatj_1,(2*Mx+1,2*Mx+1))
    tcurrent = tj
    tnext = tjj
    dt = tnext-tcurrent

    i = Mx
    l = getSpacePos(i)
    #bnoundary condition
    #pos Mx, Mx
    triMatj[l][l]=1+0.5*dt*alphaMx_J(tcurrent,spaceDiscr[l])
    triMatj_1[l][l]=1-0.5*dt*alphaMx_J(tnext,spaceDiscr[l])

    #pos Mx,Mx-1
    triMatj[l][getSpacePos(i-1)]=0.5*dt*betaMx_J(tcurrent,spaceDiscr[l]) 
    triMatj_1[l][getSpacePos(i-1)]=-0.5*dt*betaMx_J(tnext,spaceDiscr[l])
    
    i = -Mx
    l = getSpacePos(i)
    #pos -Mx,-Mx+1
    triMatj[l][getSpacePos(i+1)]=0.5*dt*theta_Mx_J(tcurrent,spaceDiscr[l])
    triMatj_1[l][getSpacePos(i+1)]=-0.5*dt*theta_Mx_J(tnext,spaceDiscr[l])
    
    #pos -Mx, -Mx
    triMatj[l][l]= 1+0.5*dt*kappa_Mx_J(tcurrent,spaceDiscr[l])
    triMatj_1[l][l]= 1-0.5*dt*kappa_Mx_J(tnext,spaceDiscr[l])
    
    i = Mx-1
    while i >= -Mx+1:
        l = getSpacePos(i)
        #print(L)
        #coeff i+1,j and coeff i+1,j+I
        if(i+1<=Mx):
            triMatj[l][getSpacePos(i+1)]=coeffIPlusOne_J(tcurrent,spaceDiscr[l])
            triMatj_1[l][getSpacePos(i+1)]=coeffIPlusOne_J(tnext,spaceDiscr[l])
        
        triMatj[l][l]=coeffI_J(tcurrent,spaceDiscr[l])
        triMatj_1[l][l]=coeffI_J(tnext,spaceDiscr[l])

        #coeff i-1,j and coeff i-1,j+1
        if(i-1>=-Mx):
            triMatj[l][getSpacePos(i-1)]=coeffIMinusOne_J(tcurrent,spaceDiscr[l]) 
            triMatj_1[l][getSpacePos(i-1)]=coeffIMinusOne_J(tnext,spaceDiscr[l])
        i = i-1
        
    return (triMatj,triMatj_1)

def edpSolverRNMeasure():
    #ResoLution EDP
    #nextVaLues = np.zeros(2*Mx+1)
    #CurrentVaLues =np.zeros(2*Mx+1)
    priceInTime = np.zeros(N)
    #iniiaLisation of final value
    print("Percentage %")
    print("0%")
    #i = Mx
    t= timeDisc[N-1]
    lenB = len(bermExcersise)
    rtmp = pd.DataFrame(spaceDiscr)
    nextValues = rtmp.apply(lambda u : np.maximum(getVSwap(t,lenB-1,u),0)).values[:,0]
    #print(type(nextVaLues))
    #print(t)
    #whiLe i >= -Mx:
    #nextVaLues[getSpacePos(i)]= np.maximum(0,getVSwap(t,LenB-1,spaceDiscr[getSpacePos(i)]))
    #i = i-1
    priceInTime[N-1] = nextValues[getSpacePos(0)]
    #print(nextVaLues)
    
    k = N-2
    while k >=0:
        if ((N-2-k)%((int)((N-2)/10))) == 0:
            print(str(round(((N-2-k)/(N-2))*100,2))+"%")
        
        tnext = timeDisc[k+1]
        tcurrent = timeDisc[k]
        #print("tcurrent....")
        #print(tcurrent)
        # print("tnext ....")
        #print(tnext)
        dt = tnext-tcurrent
        t = tcurrent
        #resoLution crancK nichoLson
        #fiLLing matrix
        ##caLcuL di
        #print(triMatj)
        #print(triMatj_1)
        A_T =getA(t,tnext)
        tmp = np.eye(2*Mx+1)-0.5*dt*A_T[1]
        d = np.matmul(tmp,nextValues)
        
        tmp = np.eye(2*Mx+1)+0.5*dt*A_T[0]
        invMM = np.linalg.inv(tmp)
        CurrentValues = np.matmul(invMM,d)
        ####check if tcurrent in Excersise time
        if t in bermExcersise:
        #print(rtmp)
            vIntrBis = rtmp.apply(lambda u : np.maximum(getVSwap(t,np.where(bermExcersise == t)[0][0],u),0)).values[:,0]        
            CurrentValues = np.maximum(CurrentValues,vIntrBis)
        
        nextValues = CurrentValues
        priceInTime[k] = CurrentValues[getSpacePos(0)]
        k = k-1
        
    #print("grid vaLues 	
    #print(CurrentVaLues)
    #print("....end Grid VaLues...")
    print("profile of price")
    plt.plot(priceInTime)
    plt.show()
    print("100%")
    print("end percentage")
    return CurrentValues[getSpacePos(0)]

priceUSD = edpSolverRNMeasure()



Percentage %
0%


IndexError: index 0 is out of bounds for axis 0 with size 0

#### Edp FwdMeasure Solver

In [142]:
###Settings coeff for cranc Nicholson
#coeff i+1, j
def coeffIPlusOne_J(tj,x):
    vol = sig(tj,x)
    res = -((b(tj,x)/(2*dx))+(vol*vol/(2*dx*dx)))
    return res

#coeff i,j
def coeffI_J(tj,x):
    vol = sig(tj,x)
    res = ((vol*vol/(dx*dx)))
    #print(res)
    return res

#coeff i-1,j
def coeffIMinusOne_J(tj,x):
    vol = sig(tj,x)
    res = ((b(tj,x)/(2*dx))-(vol*vol/(2*dx*dx)))
    # print(res)
    return res

def alphaMx_J(tj,x):
    return 2*coeffIPlusOne_J(tj,x)+coeffI_J(tj,x)

def betaMx_J(tj,x):
    return coeffIMinusOne_J(tj,x)-coeffIPlusOne_J(tj,x)

def theta_Mx_J(tj,x):
    return coeffIPlusOne_J(tj,x)-coeffIMinusOne_J(tj,x)

def kappa_Mx_J(tj,x):
    return 2*coeffIMinusOne_J(tj,x)+coeffI_J(tj,x)

def getA(tj,tjj):
    triMatj = [0.0]*(2*Mx+1)*(2*Mx+1)
    triMatj = reshape(triMatj,(2*Mx+1,2*Mx+1))
    triMatj_1 = [0.0]*(2*Mx+1)*(2*Mx+1)
    triMatj_1 = reshape(triMatj_1,(2*Mx+1,2*Mx+1))
    tcurrent = tj
    tnext = tjj
    dt = tnext-tcurrent

    i = Mx
    l = getSpacePos(i)
    #bnoundary condition
    #pos Mx, Mx
    triMatj[l][l]=1+0.5*dt*alphaMx_J(tcurrent,spaceDiscr[l])
    triMatj_1[l][l]=1-0.5*dt*alphaMx_J(tnext,spaceDiscr[l])

    #pos Mx,Mx-1
    triMatj[l][getSpacePos(i-1)]=0.5*dt*betaMx_J(tcurrent,spaceDiscr[l]) 
    triMatj_1[l][getSpacePos(i-1)]=-0.5*dt*betaMx_J(tnext,spaceDiscr[l])
    
    i = -Mx
    l = getSpacePos(i)
    #pos -Mx,-Mx+1
    triMatj[l][getSpacePos(i+1)]=0.5*dt*theta_Mx_J(tcurrent,spaceDiscr[l])
    triMatj_1[l][getSpacePos(i+1)]=-0.5*dt*theta_Mx_J(tnext,spaceDiscr[l])
    
    #pos -Mx, -Mx
    triMatj[l][l]= 1+0.5*dt*kappa_Mx_J(tcurrent,spaceDiscr[l])
    triMatj_1[l][l]= 1-0.5*dt*kappa_Mx_J(tnext,spaceDiscr[l])
    
    i = Mx-1
    while i >= -Mx+1:
        l = getSpacePos(i)
        #print(L)
        #coeff i+1,j and coeff i+1,j+I
        if(i+1<=Mx):
            triMatj[l][getSpacePos(i+1)]=coeffIPlusOne_J(tcurrent,spaceDiscr[l])
            triMatj_1[l][getSpacePos(i+1)]=coeffIPlusOne_J(tnext,spaceDiscr[l])
        
        triMatj[l][l]=coeffI_J(tcurrent,spaceDiscr[l])
        triMatj_1[l][l]=coeffI_J(tnext,spaceDiscr[l])

        #coeff i-1,j and coeff i-1,j+1
        if(i-1>=-Mx):
            triMatj[l][getSpacePos(i-1)]=coeffIMinusOne_J(tcurrent,spaceDiscr[l]) 
            triMatj_1[l][getSpacePos(i-1)]=coeffIMinusOne_J(tnext,spaceDiscr[l])
        i = i-1
        
    return (triMatj,triMatj_1)


def edpSolverFwdMeasure():
    #ResoLution EDP
    #nextVaLues = np.zeros(2*Mx+1)
    #CurrentVaLues =np.zeros(2*Mx+1)
    priceInTime = np.zeros(N)
    #iniiaLisation of final value
    print("Percentage %")
    print("0%")
    #i = Mx
    t= timeDisc[N-1]
    lenB = len(bermExcersise)
    rtmp = pd.DataFrame(spaceDiscr)
    nextValues = rtmp.apply(lambda u : np.maximum(getVSwap(t,lenB-1,u)/getDF(t,u,MeasureT),0)).values[:,0]
    
    priceInTime[N-1] = nextValues[getSpacePos(0)]
    
    k = N-2
    while k >=0:
        if ((N-2-k)%((int)((N-2)/10))) == 0:
            print(str(round(((N-2-k)/(N-2))*100,2))+"%")
        
        tnext = timeDisc[k+1]
        tcurrent = timeDisc[k]        
        dt = tnext-tcurrent
        t = tcurrent
        #resoLution crancK nichoLson
        #fiLLing matrix
        ##caLcuL di
        #print(triMatj)
        #print(triMatj_1)
        A_T = getA(t,tnext)
        tmp = np.eye(2*Mx+1)-0.5*dt*A_T[1]
        d = np.matmul(tmp,nextValues)
        
        tmp = np.eye(2*Mx+1)+0.5*dt*A_T[0]
        invMM = np.linalg.inv(tmp)
        CurrentValues = np.matmul(invMM,d)
        
        ####check if tcurrent in Excersise time
        if t in bermExcersise:
        #print(rtmp)
            vIntrBis = rtmp.apply(lambda u : np.maximum(getVSwap(t,np.where(bermExcersise == t)[0][0],u)/getDF(t,u,MeasureT),0)).values[:,0]
            CurrentValues = np.maximum(CurrentValues,vIntrBis)
            
        nextValues = CurrentValues
        priceInTime[k] = CurrentValues[getSpacePos(0)]
        k = k-1
        #print("grid vaLues 	")
        #print(CurrentVaLues)
        #print("....end Grid VaLues... o)
    print("profile of price")
    plt.plot(priceInTime)
    plt.show()
    print("100%")
    print("end percentage")
    return CurrentValues[getSpacePos(0)]*getDF_d(MeasureT)

priceUSD = edpSolverFwdMeasure()

#### American MC

In [ ]:
####DiffUsion xt under forward measure T
#MC Param
NbSimul = 250000
NbRegression = 3
HorizonSimul = np.max(bermExcersise)
dt = yearfraction("100Y")
N = (int)(HorizonSimul/dt)
dtReliquat = HorizonSimul-N*dt

if dtReliquat > 0:
    N = N+2
else:
    N = N+1
    
timeDisc = np.zeros(N)
k = N-1
while k > 0:
    timeDisc[k] = HorizonSimul-(N-1-k)*dt
    k = k-1
    
listTime = timeDisc.tolist()

for el in bermExcersise: 
    listTime.append(el)
    
listTime. sort()
listTime = list(set(listTime))
listTime. sort()
timeDisc = np.asarray(listTime)
N = len(timeDisc)

In [ ]:
def simulateX():
    sizeT = N
    matRes = [0.0]*sizeT*NbSimul
    matRes = reshape(matRes,(NbSimul,sizeT))
    
    j = 0
    while j < N-1:
        vZ = np.random.randn(NbSimul)
        t = timeDisc[j]
        dt = timeDisc[j+1]-t
        std = np.sqrt(getVariance(t,t+dt))
        matRes[:,j+1] = matRes[:,j]*np.exp(-L*dt)+std*vZ
        j = j+1
    return matRes

def getDiscountedCash(ti_pos,tj_pos,scenariok,MatX):
    sum = 0
    #if ((ftoat)(ti_pos) in timeDisc) and ((ftoat)(tj_pos) in timeDisc):
    if scenariok <NbSimul:
        res = MatX[scenariok,ti_pos:(tj_pos+1)]
        i = 0
        while i < len(res)-1:
            dt = timeDisc[ti_pos+i+1]-timeDisc[ti_pos+i]
            sum = sum + 0.5*(ft(timeDisc[ti_pos+i])+res[i]+res[i+1]+ft(timeDisc[ti_pos+i+1]))*dt
            i = 1+1
    return np.exp(-sum)
    print("erreur")
    

def getDiscountedCashV2(ti_pos,tj_pos,MatX):
    VectRes = np.zeros((NbSimul,1))    
    Nb = len(MatX[0,ti_pos:(tj_pos+1)])
    i = 0
    while i < Nb-1:
        dt = timeDisc[ti_pos+i+1]-timeDisc[ti_pos+i]
        VectRes[:,0] = VectRes[:,0] + 0.5*(ft(timeDisc[ti_pos+i])+MatX[:,ti_pos]+q+MatX[:,ti_pos+i+1]+ft(timeDisc[ti_pos+i+1]))*dt
    i = 1+1
    VectRes = np.exp(-VectRes)
    #print(VectRes)
    return VectRes.flatten()
    print("erreur")


In [144]:
def laguerrePolynomialV(x,rank):
    if rank == 0:
    #return x
        return np.exp(-x*0.5)
    if rank == 1:
    #return x*x
        return laguerrePolynomial(x,0)*(1-x)
    if rank == 2:
    #return x*x*x
        return laguerrePolynomial(x,0)*(1-2*x+(x*x*0.5))
    if rank == 3:
        return laguerrePolynomial(x,0)*(-(x**3)+(9*x**2)-18*x+6)*(1/6)
    if rank == 4:
        return laguerrePolynomial(x,0)*((x**4)-16*(x**3)+72*(x**2)-96*x+24)*(1/24)
    if rank == 5:
        return laguerrePolynomial(x,0)*(-(x**5)+25*(x**4)-200*(x**3)+600*(x**2)-600*x+120)*(1/120)
    if rank == 6:
        return laguerrePolynomial(x,0)*((x**6)-36*(x**5)+450*(x**4)-2400*(x**3)+5400*(x**2)-4320*x+720)*(1/720)
    return 0.0

In [146]:
def laguerrePolynomial(x,rank):
    return x**rank

def ContinuousValue(coeffAlpha,NbRegression,x):
    return np.sum((applyLaguerreToX(NbRegression,x)*coeffAlpha))

def applyLaguerreToX(NbRegression,x):
    #print(NbRegression)
    return [laguerrePolynomial(x,i) for i in range((int)(NbRegression))]


## Vt and Xt are arrays
def regressor(Vt,Xt,t,posStartSwap):
    N = len(Xt)
    N0 = N
    if NbRegression < 1 : 
        print("erreur") 
        return 0
    
    if len(Vt) == len(Xt):    
        i = 0
        k = 0
        tmp = pd.DataFrame(Xt)
        IntrValue = tmp.apply(lambda x : np.maximum(getVSwap(t,posStartSwap,x),0)).values[:,0]
        #print("	Intrinseque vaLue ....")
        #print(IntrVaLue)
        #print("	Intrinseque vaLue End ....")
        uA = np.array(np.where(IntrValue>0.0)).flatten()
        subX = Xt[uA]
        #print(u########### sub X before Normalisation ########")
        #print(Len(subX))
        #print("###########End sub X before NormaLisation ########")
        if len(subX) > 0:
            meanX = np.mean(subX)
            varX = np.var(subX)
            subX = (subX - meanX)/np.sqrt(varX)
            #print("########### sub X After NormaLisation ########")
            #print(Len(subX))
            #print("###########End sub X After NormaLisation ########")
        Vtmp = Vt[uA].reshape(len(subX),1)
        #print(Vtmp)
        N_0 = len(subX)
        sizeCoeff = NbRegression
        M = np.ones((sizeCoeff,N_0))
        if len(subX) == 0:
            return (np.zeros(NbRegression),IntrValue,uA,M)
        if len(subX) <= NbRegression:
            return (np.zeros(NbRegression),IntrValue,uA,M)
        
        while i < len(subX):
            M[:,i] = applyLaguerreToX(NbRegression,subX[i])
            i = 1+1
        #print("########### M ########")
        #print(M.shape)
        #print("###########End M########")
        trsM = M.transpose()
        tmp = np.matmul(M,trsM)
        
        ## reguLarisation
        ridgeLambda = 0.0
        ridgeLxl = np.eye(sizeCoeff)*ridgeLambda
        tmp = tmp + ridgeLxl
        
        ##end ReguLarisation
        print("########### Covariance ########")
        print(tmp)
        print("########### End Covariance ########")
        invMM = np.linalg.inv(tmp)
        #tmp = np.matmuL(trsM,invMM)
        #coeffALpha = np.matmuL(Vtmp,tmp)
        tmp = np.matmul(invMM,M)
        coeffAlpha = np.matmul(tmp,Vtmp)
        #print("shape of aLpha " str(coeffALpha.shape))
        print("coefff alpha ")
        print(coeffAlpha)
        print("End coefff alpha ")
        return (coeffAlpha,IntrValue,uA,M)
        print("erreur")

In [ ]:
Sim_XX1 = simulateX()

In [147]:
def americanMC():
    x =Sim_XX1
    CFs = [0.0]*(len(bermExcersise)*NbSimul)
    CFs = reshape(CFs,(NbSimul,len(bermExcersise)))
    tnextIndice = 0
    
    k = len(bermExcersise)-1
    N= len(bermExcersise)
    
    Tn = bermExcersise[k]
    indiceTn = np.where(timeDisc == Tn)[0][0]
    tnextIndice = indiceTn
    Xlast = x[:,indiceTn]
    
    #CFs[:,le] = np.fromiter((np.maximum(getV5wap(Tn,Tn,xi,sens),0) for xi in XLast), XLast.dtype, count=NbSimuL)
    rtmp = pd.DataFrame(Xlast)
    CFs[:,k] = rtmp.apply(lambda u : np.maximum(getVSwap(Tn,k,u),0)).values[:,0]
    
    IndexExercise = [-1]*(NbSimul)
    IndexExercise = reshape(IndexExercise,(NbSimul,1))
    
    dTmp = np.array(np.where(CFs[:,k]>0.0)).flatten()
    IndexExercise[dTmp]= k
    #print(N)
    k = k-1
    while k >= 0:
        if (int)((N-2)/10)> 0:
            if ((N-2-k)%((int)((N-2)/10))) == 0:
                print(str(round(((N-2-k)/(N-2))*100,2))+"%")
        elif N > 2:
            print(str(round(((N-2-k)/(N-2))*100,2))+"%")
        #print("Ok")
        #find x vector
        
        t = bermExcersise[k]
        indicet = np.where(timeDisc == t)[0][0]
        xvect = x[:,indicet]
        #print("indicet && t ")
        #print(str(indicet)* "("+str(t))
        #print("	End indice && t	
        #print("	Xvect 	")
        #print(xvect)
        #print("	End Xvect 	")
        #print(pd.DataFrame(xvect))
        CFDis = getDiscountedCashV2((indicet),(tnextIndice),x)*CFs[:,k+1]
        tuAlpha = regressor(CFDis,xvect,t,k)
        
        coeffAlpha = tuAlpha[0]
        IntrValue = tuAlpha[1]
        
        #print(IntrVaLue)
        err = 0
        SSE = 0
        SST = 0
        NumberR2 = 0
        meanY_2 = 0
        meanY = 0
        
        posIntrValue = IntrValue[IntrValue>0.0]
        uA = tuAlpha[2]
        M = tuAlpha[3]
        
        #vectCVaLue = ttttv.appLy(Lambda xit : np.maximum(ContinuousVaLue(coeffALpha, NbRegression,xit),0)).vaLues[:,0]
        #print("Continuous vaLue")
        #vectCVaLue = np.maximum(np.matmuL(coeffALpha,h),0)
        vectCValue = np.matmul(coeffAlpha.transpose(),M)
        #print("	End continuous VaLue ")
        #print("	DvaLue 	 ,,)
        dValueAtk = (np.maximum(posIntrValue,vectCValue))*(posIntrValue>vectCValue)
        #print(dVdLueAtk)
        #print("	End DvaLue 	
        ########heeeeeeeere ############
        print("..................................")

        CFs[uA,k] = dValueAtk
        posExces = np.where(CFs[:,k]>0)
        CFs[posExces,k+1:] = 0
        #CFs[uA,k+1:] = 0
        #print(CFs[:,k])
        dTmp = np.array(np.where(CFs[:,k]>0.0)).flatten()
        IndexExercise[dTmp]= k
        tnextIndice=indicet
        k = k-1
        
    print("...................")
    print("cashflows")
    print(pd.DataFrame(CFs))
    print(" end cashflows")
    print("Index Exercise")
    print(IndexExercise)
    # print("Discounted Cash
    #print(matDiscountE)
    print("...........")
    price = 0
    mean2 = 0
    
    for i in range(N):
        t = bermExcersise[i]
        vectprice_i = getDiscountedCashV2(0,np.where(timeDisc== t)[0][0],x)*CFs[:,i]
        price = price + np.sum(vectprice_i)
        mean2 = mean2 + np.sum(vectprice_i**2)
        
    price = price/NbSimul
    stdDev = (mean2/NbSimul)-price*price
    qlte = 2.32634787404084 #99% quantiLe
    IC_ = price - qlte*np.sqrt(stdDev/Nb5imul)
    IC = price + qlte*np.sqrt(stdDev/NbSimul)
    
    # print("	 n)	
    #print("Index Exerc")	
    #print(pd.DataFrame(IndexExercise))	
    #print("	Price	 ")	
    #print(price)	
    return (price,(IC_,IC))


In [ ]:
#FX
ccyCCYUsd = 1.30685
targetPrice = 378427.76
price = americanMC()

In [ ]:
priceUSD = price[0]
print("price")
print(priceUSD)
print("IC : ( "+str(price[1][0]*ccyCCYUsd)+","+ str(price[1][1]*ccyCCYUsd)+ " )")